In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fftfreq, rfft, irfft, rfftfreq

# Load and center monthly Sunspots data

In [ ]:
df_sunspots = pd.read_csv('Sunspots_observations.csv', skiprows=6)

t_data = df_sunspots['Year CE.1'].dropna().to_numpy()
data = df_sunspots['SNm'].dropna().to_numpy()

In [ ]:
data_mean = np.mean(data)
data -= data_mean
print(data)

# Finding and removing peaks iteratively

In [ ]:
def find_and_remove_peaks(data, t, threshold):
    # Compute the Fourier transform of the data
    fft_data = rfft(data)
    frequencies = rfftfreq(len(t), np.mean(np.diff(t)))

    # Find the dominant frequency with the highest amplitude
    dominant_index = np.argmax(np.abs(fft_data))
    dominant_frequency = frequencies[dominant_index]
    
    # Calculate the magnitude of the dominant peak
    magnitude = 2.0 * np.abs(fft_data[dominant_index]) / len(t)

    # If the magnitude of the dominant peak is above the threshold, remove it from the data
    if magnitude > threshold:
        # Get the period, phase, B1 and B2 of the dominant peak
        period = 1 / np.abs(dominant_frequency)
        phase = (np.angle(fft_data[dominant_index]) - 2 * np.pi / period * t.min()) % (2 * np.pi)
        B1 = magnitude * np.cos(phase)
        B2 = -magnitude * np.sin(phase)

       # Set the dominant frequency component to zero
        fft_data[dominant_index] = 0
        
        # Reconstruct the data after removing the dominant peak
        data_without_dominant = irfft(fft_data, n=len(data))
        return data_without_dominant, magnitude, period, phase, B1, B2
    else:
        return data, None, None, None, None, None

In [ ]:
# Set the threshold value to control which peaks to remove
threshold = 5  # Adjust this threshold value as needed

# Create a copy of the data to work with
data_cleaned = data.copy()
magnitudes = []
periods = []
phases = []
B1_sequence = []
B2_sequence = []

In [ ]:
# Iteratively remove peaks based on the threshold
counter = 0

while True:
    data_cleaned, magnitude, period, phase, B1, B2 = find_and_remove_peaks(data_cleaned, t_data, threshold)
    counter += 1
    if magnitude is None or counter == 16:
        break  # No more significant peaks to remove
    magnitudes.append(magnitude)
    periods.append(period)
    phases.append(phase)
    B1_sequence.append(B1)
    B2_sequence.append(B2)

In [ ]:
# Print the extracted values for each significant peak
print("Extracted Magnitudes:", magnitudes)
print("Extracted Periods:", periods)
print("Extracted Phases:", phases)
print("B1 values", B1_sequence)
print("B2 values", B2_sequence)

In [ ]:
# Plot the original data and the data after removing significant peaks
plt.figure(figsize=(10, 6))
plt.plot(t_data, data, label="Original Data", lw=2)
plt.plot(t_data, data_cleaned, label="Data After Removing Peaks", lw=2)
plt.xlabel("Time (months)")
plt.ylabel("Sunspot Data")
plt.legend()
plt.show()

# Generating model predictions and estimating the standard deviation of the noise

In [ ]:
# Generate model predictions based on the stored parameters
def generate_model_predictions(data, t_data, magnitudes, periods, phases):
    model_predictions = np.zeros_like(data)

    for A, T, phi in zip(magnitudes, periods, phases):
        component = A * np.cos(2 * np.pi * t_data / T  + phi)
        model_predictions += component

    return model_predictions    

Estimate the standard deviation of the noise

In [ ]:
# Calculate the residuals (observed data - model predictions)
model_predictions = generate_model_predictions(data, t_data, magnitudes, periods, phases)
residuals = data - model_predictions

# Estimate the noise standard deviation from the residuals
estimated_noise_std = np.std(residuals)

print("Estimated Noise Standard Deviation:", estimated_noise_std)

Plot the model predictions and the original data for comparison

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(t_data, data, label="Original Data", lw=2)
plt.plot(t_data, model_predictions, label="Model predictions", lw=2)
plt.xlabel("Time (months)")
plt.ylabel("Sunspot Data")
plt.legend()
plt.show()

Plot the residuals

In [ ]:
plt.plot(t_data, residuals, label="Residuals", lw=2)
plt.show()

The code below is used for checking the parameter values (if everything is okay)

In [ ]:
for i in range(15):
    print(f"Iteration {i + 1}")
    print(f"Period: {periods[i]}")
    print(f"Magnitude: {magnitudes[i]}")
    print(f"Phase: {phases[i]}")
    print(f"B1: {B1_sequence[i]}, B2: {B2_sequence[i]}")
    print("\n")

In [ ]:
print(len(magnitudes))